In [9]:
import pandas as pd
from tqdm import tqdm
import re
from deep_translator import GoogleTranslator

file_path = 'D:\PROJEK HOTEL\Projek-Analisis-Ulasan-Hotel\DatasetHotel\BUMN\Bintang3\KHASMalioboroHotel.csv'
df = pd.read_csv(file_path)

<>:6: SyntaxWarning: invalid escape sequence '\P'
<>:6: SyntaxWarning: invalid escape sequence '\P'
C:\Users\Asus Vivobook\AppData\Local\Temp\ipykernel_18980\3256940559.py:6: SyntaxWarning: invalid escape sequence '\P'
  file_path = 'D:\PROJEK HOTEL\Projek-Analisis-Ulasan-Hotel\DatasetHotel\BUMN\Bintang3\KHASMalioboroHotel.csv'


In [10]:
df.head()

,No,User Name,Review Count,Review Time,Rating,Review Text
0,1.0,Mckatog DP,36,2025-12-18,5,Hotel bintang 3 harga bintang 4... tapi sangat...
1,2.0,Endi Sitompul,119,2025-10-08,5,I stayed 1 night only with my colleagues and h...
2,3.0,Fariez Nurkholiq Ashshiddiqie,138,2025-12-08,4,"Hotelnya bagus, pelayanan ramah, suka karena d..."
3,4.0,aurelia valen,11,2026-01-01,3,"Hotelnya over all bgus, lokasi juga deket seka..."
4,5.0,sarah,22,2025-12-18,5,"first time ke jogja inap di hotel Khas, untuk ..."


In [11]:
# DROP KOLOM TIDAK PERLU
# hapus 'No' dan 'Review Time' 
cols_to_drop = ['No', 'Review Time']
# Gunakan list comprehension agar tidak error kalau kolomnya tidak ditemukan
df = df.drop(columns=[c for c in cols_to_drop if c in df.columns])

# DATA CLEANING
slang_dict = {
    'yg': 'yang', 'ga': 'tidak', 'gak': 'tidak', 'nggak': 'tidak', 
    'tp': 'tapi', 'krn': 'karena', 'utk': 'untuk', 'sdh': 'sudah', 
    'udh': 'sudah', 'blm': 'belum', 'dgn': 'dengan', 'dlm': 'dalam', 
    'bgt': 'banget', 'tdk': 'tidak', 'jgn': 'jangan', 'krg': 'kurang', 
    'sy': 'saya', 'ak': 'aku', 'kalo': 'kalau', 'kl': 'kalau', 
    'dr': 'dari', 'bs': 'bisa', 'kmn': 'kemana', 'tmn': 'teman',
    'bgs': 'bagus', 'dtg': 'datang', 'br': 'baru', 'ok': 'oke',
    'thx': 'terima kasih', 'makasih': 'terima kasih', 'tks': 'terima kasih',
    'min': 'minus', 'plus': 'tambah', 'chek': 'check', 'chekout': 'check out',
    'chekin': 'check in', 'in': 'masuk', 'out': 'keluar', 'pas': 'saat',
    'pd': 'pada', 'pake': 'pakai', 'sm': 'sama', 'lbh': 'lebih', 
    'bkn': 'bukan', 'spt': 'seperti', 'jd': 'jadi', 'aja': 'saja', 
    'aj': 'saja', 'kmr': 'kamar', 'kmar': 'kamar', 'mnt': 'minta', 
    'dl': 'dulu', 'skrg': 'sekarang', 
    'dg': 'dengan', 'yk': 'yogyakarta'
}

def clean_text_advanced(text):
    if not isinstance(text, str): return ""
    text = text.lower()
    text = re.sub(r'(.)\1{2,}', r'\1', text) 
    text = re.sub(r'[^\w\s]', ' ', text)     
    words = text.split()
    normalized_words = [slang_dict.get(word, word) for word in words]
    text = ' '.join(normalized_words)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [12]:
#TRANSLATE INDONESIA
def translate_to_indonesia(text):
    if not text or len(text) < 3: return text
    try:
        # Menghandle otomatis deteksi bahasa dan translate ke 'id' (Indonesia)
        translated = GoogleTranslator(source='auto', target='id').translate(text)
        return translated
    except Exception as e:
        return text

In [13]:
#Ekseskusi
# Drop kolom dan Cleaning awal
df = df.drop(columns=[c for c in ['No', 'Review Time'] if c in df.columns])
df['Review Text Cleaned'] = df['Review Text'].apply(clean_text_advanced)

# Langkah Tambahan: Translate
# Menggunakan tqdm agar Anda bisa melihat progress di Intel CPU
tqdm.pandas(desc="🌐 Menerjemahkan ke Indonesia")
df['Review Text Cleaned'] = df['Review Text Cleaned'].progress_apply(translate_to_indonesia)

# Finalisasi DataFrame
df = df.drop(columns=['Review Text'])
df = df.rename(columns={'Review Text Cleaned': 'Review Text'})
df = df.drop_duplicates().dropna(subset=['Review Text'])

# Simpan
output_filename = 'HarperMangkubumi-Yogyakarta_BERSIH_TRANSLATED.csv'
df.to_csv(output_filename, index=False)

print(f"\n✅ SELESAI! File disimpan: {output_filename}")
print(df.head())

🌐 Menerjemahkan ke Indonesia: 100%|██████████| 473/473 [06:21<00:00,  1.24it/s] 


✅ SELESAI! File disimpan: HarperMangkubumi-Yogyakarta_BERSIH_TRANSLATED.csv
                       User Name  Review Count  Rating  \
0                     Mckatog DP            36       5   
1                  Endi Sitompul           119       5   
2  Fariez Nurkholiq Ashshiddiqie           138       4   
3                  aurelia valen            11       3   
4                          sarah            22       5   

                                         Review Text  
0  hotel bintang 3 harga bintang 4 tapi sangat ma...  
1  Saya menginap 1 malam hanya dengan rekan-rekan...  
2  hotelnya bagus pelayanan ramah suka karena dis...  
3  hotelnya over all bgus lokasi juga deket sekal...  
4  first time ke jogja inap di hotel khas untuk s...  


In [14]:
import os

# 1. Tentukan folder tujuan (Sesuaikan dengan folder proyek Anda)
# Gunakan 'r' di depan path agar backslash Windows tidak error
folder_hasil = r"D:\PROJEK HOTEL\Projek-Analisis-Ulasan-Hotel\Result"

# Buat foldernya jika belum ada
if not os.path.exists(folder_hasil):
    os.makedirs(folder_hasil)

# 2. Gabungkan folder dengan nama file
nama_file_final = "HarperMangkubumi_BERSIH_TRANSLATED.csv"
full_path_simpan = os.path.join(folder_hasil, nama_file_final)

# 3. Proses Simpan
try:
    # Simpan ke CSV
    df.to_csv(full_path_simpan, index=False, encoding='utf-8-sig')
    
    # Simpan juga ke Excel sebagai cadangan untuk laporan (lebih mudah dibuka di Excel)
    full_path_excel = full_path_simpan.replace('.csv', '.xlsx')
    df.to_excel(full_path_excel, index=False)
    
    print(f"\n✅ BERHASIL DISIMPAN!")
    print(f"📍 Lokasi CSV: {full_path_simpan}")
    print(f"📍 Lokasi Excel: {full_path_excel}")

except Exception as e:
    print(f"❌ Gagal menyimpan file: {e}")


✅ BERHASIL DISIMPAN!
📍 Lokasi CSV: D:\PROJEK HOTEL\Projek-Analisis-Ulasan-Hotel\Result\HarperMangkubumi_BERSIH_TRANSLATED.csv
📍 Lokasi Excel: D:\PROJEK HOTEL\Projek-Analisis-Ulasan-Hotel\Result\HarperMangkubumi_BERSIH_TRANSLATED.xlsx
